In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.6 MB/s eta 0:00:00


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.svm import SVC
from scipy.stats import uniform
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [5]:
train = pd.read_csv('/content/drive/MyDrive/microsoft-x-data-science-nigeria-2024-ai-bootcamp-qualification-hackathon20240918-4983-287w11/Train Dataset .csv')
test = pd.read_csv('/content/drive/MyDrive/microsoft-x-data-science-nigeria-2024-ai-bootcamp-qualification-hackathon20240918-4983-287w11/Test Dataset.csv')
ss = pd.read_csv('/content/drive/MyDrive/microsoft-x-data-science-nigeria-2024-ai-bootcamp-qualification-hackathon20240918-4983-287w11/Sample Submission.csv')

In [6]:
train.shape

(7303, 15)

In [7]:
train.head()

,Id,Age,Sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,16167,33,0,1,158,205,1,0,154,0,1.5,1,4,1,1
1,11275,53,1,2,198,154,0,1,104,0,0.8,2,1,0,0
2,13251,37,1,2,101,202,1,0,155,0,2.1,1,3,1,1
3,19921,75,0,0,113,306,1,2,88,1,4.9,0,2,2,1
4,11293,35,1,2,139,419,1,1,166,1,0.9,2,4,0,1


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7303 entries, 0 to 7302
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Id        7303 non-null   int64  
 1   Age       7303 non-null   int64  
 2   Sex       7303 non-null   int64  
 3   cp        7303 non-null   int64  
 4   trestbps  7303 non-null   int64  
 5   chol      7303 non-null   int64  
 6   fbs       7303 non-null   int64  
 7   restecg   7303 non-null   int64  
 8   thalach   7303 non-null   int64  
 9   exang     7303 non-null   int64  
 10  oldpeak   7303 non-null   float64
 11  slope     7303 non-null   int64  
 12  ca        7303 non-null   int64  
 13  thal      7303 non-null   int64  
 14  target    7303 non-null   int64  
dtypes: float64(1), int64(14)
memory usage: 855.9 KB


In [9]:
train.describe()

,Id,Age,Sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000,7303.000000
mean,15021.535396,53.172669,0.499658,1.502533,147.447487,342.805970,0.493085,1.013008,136.506093,0.503218,3.129851,0.991510,2.019033,1.502259,0.813501
std,2886.026080,14.185970,0.500034,1.115594,31.099538,127.291998,0.499986,0.815806,38.141966,0.500024,1.791160,0.817291,1.410546,1.113137,0.389535
min,10001.000000,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12521.500000,41.000000,0.000000,1.000000,120.000000,231.000000,0.000000,0.000000,104.000000,0.000000,1.600000,0.000000,1.000000,1.000000,1.000000
50%,15054.000000,53.000000,0.000000,1.000000,148.000000,341.000000,0.000000,1.000000,137.000000,1.000000,3.100000,1.000000,2.000000,1.000000,1.000000
75%,17513.500000,65.000000,1.000000,3.000000,174.000000,450.000000,1.000000,2.000000,170.000000,1.000000,4.700000,2.000000,3.000000,2.000000,1.000000
max,19998.000000,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [10]:
train.isnull().sum()

,0
Id,0
Age,0
Sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0


In [11]:
train.dtypes

,0
Id,int64
Age,int64
Sex,int64
cp,int64
trestbps,int64
chol,int64
fbs,int64
restecg,int64
thalach,int64
exang,int64


In [12]:
test.head()

,id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,16501,70,1,0,163,495,0,2,170,1,2.0,1,0,1
1,10444,61,1,0,131,238,0,2,74,1,4.9,2,2,2
2,14288,53,1,0,95,558,1,1,73,1,0.7,1,1,0
3,10409,37,0,1,178,287,0,1,192,1,5.7,1,0,0
4,17330,35,0,3,104,281,0,0,122,0,1.3,1,4,3


In [13]:
train.columns = train.columns.str.lower()

In [14]:
# Separate the features and target from the training dataset
X = train.drop(columns=['id', 'target'])
y = train['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Cat Boost Classifier

In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100, random_seed=42)

model.fit(X_train_scaled, y_train, eval_set=(X_val_scaled, y_val), early_stopping_rounds=50)

0:	learn: 0.5296676	test: 0.5248865	best: 0.5248865 (0)	total: 59.4ms	remaining: 59.4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.2568100788
bestIteration = 20

Shrink model to first 21 iterations.


In [18]:
val_predictions = model.predict(X_val_scaled)
accuracy = accuracy_score(y_val, val_predictions)
class_report = classification_report(y_val, val_predictions)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n {class_report}")

Validation Accuracy: 0.8131
Classification Report:
               precision    recall  f1-score   support

           0       0.47      0.38      0.42       260
           1       0.87      0.91      0.89      1201

    accuracy                           0.81      1461
   macro avg       0.67      0.64      0.65      1461
weighted avg       0.80      0.81      0.81      1461



In [19]:
# Make predictions on the test set
X_test = test.drop(columns=['id'])
X_test_scaled = scaler.transform(X_test)
test_predictions = model.predict(X_test_scaled)

ss['target'] = test_predictions

ss.to_csv('catboost.csv', index=False)

Support Vector Classifier

In [20]:
svc = SVC(random_state=42)

param_dist = {
    'C': uniform(0.1, 10),
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
}

random_search = RandomizedSearchCV(svc, param_distributions=param_dist, n_iter=50,
                                   scoring='accuracy', cv=5)

random_search.fit(X_train_scaled, y_train)

best_params = random_search.best_params_
print(f"Best Parameters: {best_params}")

Best Parameters: {'C': 0.31694633869247524, 'gamma': 'auto', 'kernel': 'linear'}


In [21]:
svc_val_predictions = random_search.best_estimator_.predict(X_val_scaled)
accuracy = accuracy_score(y_val, svc_val_predictions)
class_report = classification_report(y_val, svc_val_predictions)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n {class_report}")

Validation Accuracy: 0.8220
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       260
           1       0.82      1.00      0.90      1201

    accuracy                           0.82      1461
   macro avg       0.41      0.50      0.45      1461
weighted avg       0.68      0.82      0.74      1461



In [22]:
# Make predictions on the test set
X_test = test.drop(columns=['id'])
X_test_scaled = scaler.transform(X_test)
svc_test_predictions = random_search.best_estimator_.predict(X_test_scaled)

ss['target'] = svc_test_predictions

ss.to_csv('SVC.csv', index=False)
